# Robustness
This code was adapted from the code written by Michael Lees and Debraj Roy

## Import libraries


In [1]:
import networkx as nx #import NetworkX
import numpy as np #import numpy for ...
#force drawing of graphs inline for ipython notebook
%matplotlib inline 
import matplotlib.pyplot as plt #import matplotlib for plotting/drawing grpahs
import matplotlib.patches as mpatches #for legends in the graph
from __future__ import unicode_literals #allow UTF characters in graph labels
import random # for random choice function
import copy # this is use for making deep copies of lists
from tqdm import tqdm #nice library for progress bars
import sys #for writing output to stderr

## Import networks

In [5]:
with open('vi-wiki-talk.csv', 'rb') as file_handle:
    next(file_handle, '')   # skip the header line (NOTE the first list in the CSV file doesn't contain an edge)
    V = nx.read_edgelist(file_handle, delimiter='\t', create_using=nx.DiGraph(), 
                         nodetype=str, data=(('time', str),), encoding="utf-8")

with open('sv-wiki-talk.csv', 'rb') as file_handle:
    next(file_handle, '')   # skip the header line (NOTE the first list in the CSV file doesn't contain an edge)
    S = nx.read_edgelist(file_handle, delimiter='\t', create_using=nx.DiGraph(), 
                         nodetype=str, data=(('time', str),), encoding="utf-8")
print "done"

done


## Attack and fail function

In [2]:
def fail(G): #a python function that will remove a random node from the graph G
    n = random.choice(G.nodes())  #pick a random node
    G.remove_node(n) # remove that random node, attached edges automatically removed.
    
def attack_degree(G): #remove node with maximum degree
    degrees = G.degree() # get dcitonary where key is node id, value is degree
    max_degree = max(degrees.values()) # find maximum degree value from all nodes
    max_keys = [k for k,v in degrees.items() if v==max_degree] #get all nodes who have the maximum degree (may be more than one)
    G.remove_node(max_keys[0]) #remove just the first node with max degree, we will remove others next
    

## Experimental parameters

In [7]:
fraction_of_nodes_to_remove = 0.95 # remove until this fraction of all original nodes are removed

# For the Vietnamese network
NetworkSize_vi = V.order()
num_removals_vi = int(fraction_of_nodes_to_remove * NetworkSize_vi) #number of nodes to remove

# For the Swedish network
NetworkSize_sv = S.order()
num_removals_sv = int(fraction_of_nodes_to_remove * NetworkSize_sv) #number of nodes to remove


## Measuring Original Network Statistics

In [ ]:
def diameter_ave_path_length(G):
    # We create our own function to do this so things are slightly faster, 
    # we can calculate diameter and avg path length at the same time
    max_path_length = 0
    total = 0.0
    for n in G: #iterate over all nodes
        path_length=nx.single_source_shortest_path_length(G, n) # generate shortest paths from node n to all others
        total += sum(path_length.values()) #total of all shortest paths from n
        if max(path_length.values()) > max_path_length: #keep track of longest shortest path we see.
            max_path_length = max(path_length.values())         
    try:
        avg_path_length = total / (G.order()*(G.order() - 1))
    except ZeroDivisionError:
        avg_path_length = 0.0
    return max_path_length, avg_path_length

def network_statistics(n):
    """n is a network
    Return diameter (d), average path length (l) and giant component size (s) """
    
    Gcc=sorted(nx.connected_component_subgraphs(n), key = len, reverse=True)
    G0=Gcc[0]
    d,l = diameter_ave_path_length(G0)
    s = float(G0.order()) / float(NetworkSize)
    return d,l,s